In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
# User Agent
# Headers set like below:
# User Agent
headers = {
    'authority': 'www.99acres.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': 'https://www.99acres.com/independent-house-in-gurgaon-ffid-page',
    'sec-ch-ua': '"Chromium";v="107", "Not;A=Brand";v="8"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/527.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}

### Extracting independent House


In [ ]:
ind_house = pd.DataFrame()

In [ ]:
import time

# Put start page number and end page number
start = 1 # Starting Page
end = 50 # End Page

csv_file = f"/content/drive/MyDrive/DSMP/Case Studies/Real estate/indpendent_house/independent_house-p-{start}-{end}.csv"
pageNumber = start
req=0
while pageNumber <= end:
    i=1
    url = f'https://www.99acres.com/independent-house-in-gurgaon-ffid-page-{pageNumber}'
    page = requests.get(url, headers=headers)
    pageSoup = BeautifulSoup(page.content, 'html.parser')
    req+=1
    for soup in pageSoup.select_one('div[data-label="SEARCH"]').select('section[data-hydration-on-demand="true"]'):

    # Extract property name and property sub-name
        try:
            property_name = soup.select_one('a.srpTuple__propertyName').text.strip()
            # Extract link
            link = soup.select_one('a.srpTuple__propertyName')['href']

        except:
            continue

        #Society
        try:
            society = soup.select_one('#srp_tuple_society_heading').text.strip()
        except:
            society=''

        # Detail Page
        time.sleep(1)
        page = requests.get(link, headers=headers)
        dpageSoup = BeautifulSoup(page.content, 'html.parser')
        req+=1
        try:
            #price Range
            price = dpageSoup.select_one('#pdPrice2').text.strip()
        except:
            price = ''

        # Area
        try:
            ratePerArea = soup.select_one('#srp_tuple_price_per_unit_area').text.strip()
        except:
            ratePerArea =''
        # Area with Type
        try:
            areaWithType = dpageSoup.select_one('#factArea').text.strip()
        except:
            areaWithType = ''
        try:
            area = soup.select_one('#srp_tuple_secondary_area').text.strip()
        except:
            area = ''

        # Configuration
        try:
            bedRoom = dpageSoup.select_one('#bedRoomNum').text.strip()
        except:
            bedRoom = ''
        try:
            bathroom = dpageSoup.select_one('#bathroomNum').text.strip()
        except:
            bathroom = ''
        try:
            balcony = dpageSoup.select_one('#balconyNum').text.strip()
        except:
            balcony = ''

        try:
            additionalRoom = dpageSoup.select_one('#additionalRooms').text.strip()
        except:
            additionalRoom = ''


        # Address

        try:
            address = dpageSoup.select_one('#address').text.strip()
        except:
            address = ''
        # Floor Number
        try:
            noOfFloor = dpageSoup.select_one('#floorNumLabel').text.strip()
        except:
            noOfFloor = ''

        try:
            facing = dpageSoup.select_one('#facingLabel').text.strip()
        except:
            facing = ''

        try:
            agePossession = dpageSoup.select_one('#agePossessionLbl').text.strip()
        except:
            agePossession = ''

        # Nearby Locations

        try:
            nearbyLocations = [i.text.strip() for i in dpageSoup.select_one('div.NearByLocation__tagWrap').select('span.NearByLocation__infoText')]
        except:
            nearbyLocations = ''

        # Descriptions
        try:
            description = dpageSoup.select_one('#description').text.strip()
        except:
            description = ''

        # Furnish Details
        try:
            furnishDetails = [i.text.strip() for i in dpageSoup.select_one('#FurnishDetails').select('li')]
        except:
            furnishDetails = ''

        # Features
        if furnishDetails:
            try:
                features = [i.text.strip() for i in dpageSoup.select('#features')[1].select('li')]
            except:
                features = ''
        else:
            try:
                features = [i.text.strip() for i in dpageSoup.select('#features')[0].select('li')]
            except:
                features = ''



        # Rating by Features
        try:
            rating = [i.text for i in dpageSoup.select_one('div.review__rightSide>div>ul>li>div').select('div.ratingByFeature__circleWrap')]
        except:
            rating = ''
        # print(top_f)

        try:
        # Property ID
            property_id = dpageSoup.select_one('#Prop_Id').text.strip()
        except:
            property_id = ''

        # Create a dictionary with the given variables
        property_data = {
        'property_name': property_name,
        'link': link,
        'society': society,
        'price': price,
        'rate' : ratePerArea,
        'area': area,
        'areaWithType': areaWithType,
        'bedRoom': bedRoom,
        'bathroom': bathroom,
        'balcony': balcony,
        'additionalRoom': additionalRoom,
        'address': address,
        'noOfFloor': noOfFloor,
        'facing': facing,
        'agePossession': agePossession,
        'nearbyLocations': nearbyLocations,
        'description': description,
        'furnishDetails': furnishDetails,
        'features': features,
        'rating': rating,
        'property_id': property_id
    }


        temp_df = pd.DataFrame.from_records([property_data])
        # print(temp_df)
        ind_house = pd.concat([ind_house, temp_df], ignore_index=True)
        i += 1

        if os.path.isfile(csv_file):
        # Append DataFrame to the existing file without header
            temp_df.to_csv(csv_file, mode='a', header=False, index=False)
        else:
            # Write DataFrame to the file with header
            temp_df.to_csv(csv_file, mode='a', header=True, index=False)
    print(f'{pageNumber} -> {i}')
    pageNumber += 1
    time.sleep(1)
    if req%4==0:
        time.sleep(10)
    if req%20==0:
        time.sleep(20)

35 -> 26
36 -> 26
37 -> 26
38 -> 26
39 -> 26
40 -> 26
41 -> 26
42 -> 26


AttributeError: ignored

In [ ]:
ind_house.to_csv(f'/content/drive/MyDrive/DSMP/Case Studies/Real estate/indpendent_house/independent_house-p{start}-{pageNumber-1}-{ind_house.shape[0]}rows.csv', index=False)

In [ ]:


def combine_csv_files(folder_path, combined_file_path):
    combined_data = pd.DataFrame()  # Create an empty DataFrame to hold the combined data

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            print('file_path')
            # Read the data from the current CSV file
            df = pd.read_csv(file_path)

            # Append the data to the combined DataFrame
            combined_data = combined_data.append(df, ignore_index=True)

            # Delete the original CSV file
            os.remove(file_path)

    # Save the combined data to a new CSV file
    combined_data.to_csv(combined_file_path, index=False)

# Example usage:
folder_path = '/content/drive/MyDrive/DSMP/Case Studies/Real estate/indpendent_house'  # Replace with the actual folder path
combined_file_path = '/content/drive/MyDrive/DSMP/Case Studies/Real estate/indpendent_house/independent-house.csv'  # Replace with the desired combined file path

combine_csv_files(folder_path, combined_file_path)


file_path


<ipython-input-5-28105d0af211>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(df, ignore_index=True)


file_path
file_path


<ipython-input-5-28105d0af211>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(df, ignore_index=True)
<ipython-input-5-28105d0af211>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(df, ignore_index=True)


In [ ]:
pd.read_csv(combined_file_path)

property_name  \
0        5 Bedroom House for sale in Sector 70A Gurgaon   
1      5 Bedroom House for sale in Sector 21A Faridabad   
2      10 Bedroom House for sale in Sushant Lok Phase 1   
3        21 Bedroom House for sale in Sector 54 Gurgaon   
4      12 Bedroom House for sale in Sushant Lok Phase 1   
...                                                 ...   
1090                 3 Bedroom House for sale in Bissar   
1091                 3 Bedroom House for sale in Bissar   
1092            8 Bedroom House for sale in DLF Phase 2   
1093  8 Bedroom House for sale in C Block Sushant Lo...   
1094      2 Bedroom House for sale in Sector 92 Gurgaon   

                                                   link  \
0     https://www.99acres.com/5-bhk-bedroom-independ...   
1     https://www.99acres.com/5-bhk-bedroom-independ...   
2     https://www.99acres.com/10-bhk-bedroom-indepen...   
3     https://www.99acres.com/21-bhk-bedroom-indepen...   
4     https://www.99acres.com/12-bhk-bedroom-indepen...   
...                                                 ...   
1090  https://www.99acres.com/3-bhk-bedroom-independ...   
1091  https://www.99acres.com/3-bhk-bedroom-independ...   
1092  https://www.99acres.com/8-bhk-bedroom-independ...   
1093  https://www.99acres.com/8-bhk-bedroom-independ...   
1094  https://www.99acres.com/2-bhk-bedroom-independ...   

                                       society       price              rate  \
0                             Bptp Visionnaire  5.25 Crore   ₹ 20,115/sq.ft.   
1                                          NaN   5.7 Crore  ₹ 105,751/sq.ft.   
2                                          NaN   2.1 Crore   ₹ 38,251/sq.ft.   
3                                          NaN     5 Crore   ₹ 43,066/sq.ft.   
4                                          NaN     3 Crore   ₹ 53,763/sq.ft.   
...                                        ...         ...               ...   
1090  Silverglades Tarudhan Valley Golf Resort   3.6 Crore   ₹ 14,815/sq.ft.   
1091  Silverglades Tarudhan Valley Golf Resort         NaN   ₹ 13,992/sq.ft.   
1092                                       NaN         NaN   ₹ 23,457/sq.ft.   
1093                                       NaN         NaN   ₹ 31,008/sq.ft.   
1094                  Signature Global City 92         NaN   ₹ 13,034/sq.ft.   

                       area                 areaWithType      bedRoom  \
0     (242 sq.m.) Plot Area  Plot area 290(242.48 sq.m.)   5 Bedrooms   
1      (50 sq.m.) Plot Area   Plot area 539(50.07 sq.m.)   5 Bedrooms   
2      (51 sq.m.) Plot Area       Plot area 61(51 sq.m.)  10 Bedrooms   
3     (108 sq.m.) Plot Area  Plot area 129(107.86 sq.m.)  21 Bedrooms   
4      (52 sq.m.) Plot Area    Plot area 62(51.84 sq.m.)  12 Bedrooms   
...                     ...                          ...          ...   
1090  (226 sq.m.) Plot Area  Plot area 270(225.75 sq.m.)   3 Bedrooms   
1091  (226 sq.m.) Plot Area                          NaN          NaN   
1092  (376 sq.m.) Plot Area                          NaN          NaN   
1093  (180 sq.m.) Plot Area                          NaN          NaN   
1094   (89 sq.m.) Plot Area                          NaN          NaN   

          bathroom       balcony  ...  \
0      4 Bathrooms  3+ Balconies  ...   
1      4 Bathrooms   2 Balconies  ...   
2     10 Bathrooms  3+ Balconies  ...   
3     21 Bathrooms  3+ Balconies  ...   
4     12 Bathrooms  3+ Balconies  ...   
...            ...           ...  ...   
1090   3 Bathrooms   2 Balconies  ...   
1091           NaN           NaN  ...   
1092           NaN           NaN  ...   
1093           NaN           NaN  ...   
1094           NaN           NaN  ...   

                                          address noOfFloor      facing  \
0       29b, Sector 70A Gurgaon, Gurgaon, Haryana  3 Floors  North-East   
1          Sector 21A Faridabad, Gurgaon, Haryana  2 Floors         NaN   
2           Sushant Lok Phase 1, Gurgaon, Haryana  5 Floors        W